# December 15th

Cool a hash algorithm, and one that is clearly spelled out let's implement it

In [24]:
function holidayhash(str)
    r = 0
    for c in str
        r = ((r + Int(c)) * 17) % 256
    end
    return r
end

holidayhash("HASH") === 52

true

Input sequence is just a big single line comma seperated this time

In [25]:
example = [
    "rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7"
]

examplepartoneresult = 1320

1320

In [26]:
function parsecontents(data)
    split(data[begin], ',', keepempty=false)
end

function partone(contents)
    sum(map(holidayhash, parsecontents(contents)))
end

partone(example) == examplepartoneresult

true

## Part 2
Thank god it's not rainbow tables

We're building some kind of hashmap. We need to start by building the component parts of the system

In [27]:
using DataStructures

struct AddInstruction
    hash::Int
    label::AbstractString
    focallength::Int
end

struct RemoveInstruction
    hash::Int
    label::AbstractString
end

Instruction = Union{AddInstruction,RemoveInstruction}

instructionparser = r"(\w+)(=|-)(\d*)"
function parseinstruction(ins::AbstractString)::Instruction
    (label, op, fl) = match(instructionparser, ins)
    if op == "-"
        RemoveInstruction(
            holidayhash(label),
            label
        )
    else
        AddInstruction(
            holidayhash(label),
            label,
            parse(Int, fl)
        )
    end
end

parseinstruction("ht=7")
parseinstruction("ht-")

function executeinmap(dict::Dict{Int,OrderedDict{AbstractString,Int}}, instruction::Instruction)
    if typeof(instruction) == RemoveInstruction && haskey(dict, instruction.hash)
        delete!(dict[instruction.hash], instruction.label)
    elseif typeof(instruction) === AddInstruction
        get!(dict, instruction.hash, OrderedDict())[instruction.label] = instruction.focallength
    end
end

function sumhash(dict::Dict{Int,OrderedDict{AbstractString,Int}})
    s = 0
    for (h, l) in dict
        box = h + 1
        slot = 1
        for (label, focallength) in l
            s += (box * slot * focallength)
            slot += 1
        end
    end
    return s
end

sumhash (generic function with 2 methods)

In [28]:
function parttwo(contents)
    hashmap = Dict{Int,OrderedDict{AbstractString,Int}}()
    for instruction in map(parseinstruction, parsecontents(contents))
        executeinmap(hashmap, instruction)
    end
    return sumhash(hashmap)
end

parttwo(example) == 145

true

## Results

In [29]:
include("./aoc.jl")

execute(15, partone, parttwo)

520500
213097
